In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

ImportError: cannot import name 'compat'

In [ ]:
ROOT_DIR = os.getcwd() + "\\Datasets\\"

In [ ]:
ROOT_DIR

In [ ]:
# Modifying Wiki Data: Filter to only include WIKI one month
start_date = '2017-01-01'
end_date = '2021-12-31'

def _mask(dataset, start_date, end_date):
    mask = (dataset.index >= start_date) & (dataset.index <= end_date)
    return dataset.loc[mask]  

In [ ]:
# BITCOIN PRICE INDEX
os.chdir(ROOT_DIR+'./BPI/')

BPI = pd.read_csv("./bpi.csv",  index_col="Date", parse_dates=True)
BPI.rename(columns={'Closing Price (USD)': 'close',
           '24h Open (USD)': 'open', '24h High (USD)': 'high', '24h Low (USD)': 'low'}, inplace=True)
BPI.drop(['Currency'], axis=1, inplace=True)

HASH = pd.read_csv("./hash-rate.csv",  index_col="Timestamp", parse_dates=True)
MINE_DIFF = pd.read_csv("./difficulty.csv",  index_col="Timestamp", parse_dates=True)
COST_PER_TRANS = pd.read_csv("./cost-per-transaction.csv", index_col="Timestamp", parse_dates=True)
OUPUT = pd.read_csv("./output-volume.csv", index_col="Timestamp", parse_dates=True)
EXTV = pd.read_csv("./estimated-transaction-volume-usd.csv", index_col="Timestamp", parse_dates=True)
TRADE_VOL = pd.read_csv("./trade-volume.csv", index_col="Timestamp", parse_dates=True)
NTRANS = pd.read_csv("./n-transactions.csv", index_col="Timestamp", parse_dates=True)

BPI = _mask(BPI, start_date, end_date)
HASH = _mask(HASH, start_date, end_date)
EXTV = _mask(EXTV, start_date, end_date)
NTRANS = _mask(NTRANS, start_date, end_date)
MINE_DIFF = _mask(MINE_DIFF, start_date, end_date)
COST_PER_TRANS = _mask(COST_PER_TRANS, start_date, end_date)
OUPUT = _mask(OUPUT, start_date, end_date)
TRADE_VOL = _mask(TRADE_VOL, start_date, end_date)

In [ ]:
# plt.plot(BPI.index, BPI["close"])

# fig = plt.figure(figsize=(10,6))
# n = 5
# BPI['min'] = BPI.iloc[argrelextrema(BPI.close.values, np.less_equal,
#                     order=n)[0]]['close']
# BPI['max'] = BPI.iloc[argrelextrema(BPI.close.values, np.greater_equal,
#                     order=n)[0]]['close']
# # Plot results
# plt.scatter(BPI.index, BPI['min'], c='r')
# plt.scatter(BPI.index, BPI['max'], c='g')
# plt.plot(BPI.index, BPI['close'])
# plt.axvline(x=BPI[BPI.index < "2017-12-24"]["close"].nlargest(1).index)
# plt.grid()
# plt.show()

In [ ]:
# MACRO
os.chdir(ROOT_DIR)

GP = pd.read_csv("./gold-price.csv",  index_col="Date", parse_dates=True)
GP.dropna(axis=1, how="all", inplace=True)
GP.drop(['Open', 'High', 'Low', 'Change %', 'Vol.'], axis=1, inplace=True)
GP.rename(columns={'Price': 'Gold price'}, inplace=True)
# conver string to int
for i in range(len(GP["Gold price"])):
    GP["Gold price"][i] = "".join(GP["Gold price"][i].split(","))
GP["Gold price"] = pd.to_numeric(GP["Gold price"])


USD_CNY = pd.read_csv("./usd-cny.csv",  index_col="Date", parse_dates=True)
USD_CNY.drop(['Open', 'High', 'Low', 'Change %'], axis=1, inplace=True)
USD_CNY.rename(columns={'Price': 'USD-CNY Price'}, inplace=True)

GP = _mask(GP, start_date, end_date)
USD_CNY = _mask(USD_CNY, start_date, end_date)

In [ ]:
# MEDIA AND INTERESTS
SVI = pd.read_csv("./svi.csv", index_col="Week", parse_dates=True)
# SVI.rename(columns={'Category: All categories':'SVI'}, inplace=True)

WIKI = pd.read_csv("./wikishark.csv", index_col="DateTime", parse_dates=True)
WIKI.rename(columns={'Bitcoin[en]':'Wikiviews'}, inplace=True)

SVI = _mask(SVI, start_date, end_date)
WIKI = _mask(WIKI, start_date, end_date)

In [ ]:
df = pd.concat([BPI,EXTV, NTRANS, HASH, MINE_DIFF, COST_PER_TRANS,
     GP, OUPUT, TRADE_VOL, USD_CNY, SVI, WIKI], axis=1)

# 7th differencing, (daily to weekly freq)
# df_resampled = df.resample('7D').mean().interpolate()
df_resampled = df
df_resampled.fillna(df_resampled.mean(), inplace=True)
df_resampled.to_csv("../complete-merged-df.csv")
df_resampled.head()

In [ ]:
print("Before sampling NaN values: ",len(df[df.isna().any(axis=1)]))
print("After sampling NaN values: ",len(df_resampled[df_resampled.isna().any(axis=1)]))

In [ ]:
df_resampled[df_resampled.isna().any(axis=1)]

In [ ]:
df_resampled.info()
df_resampled.describe()

In [ ]:
df_resampled.columns

In [ ]:
# Close price correlation matrix
k = len(df_resampled.columns)  # number of variables for heatmap
cols = df_resampled.corr().nlargest(k, 'close')['close'].index
cm = df_resampled[cols].corr()
cm.style.background_gradient(cmap='coolwarm').set_precision(2)